In [1]:
import os
import time

import tfx
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft

from tensorflow_metadata.proto.v0 import schema_pb2

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.components import CsvExampleGen
from tfx.components import SchemaGen
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components import ExampleValidator

# Artifact, Serving Model, Data Locations

In [2]:
ARTIFACT_STORE = os.path.join(os.sep, 'Users', 'CarlosMonsivais', 'Desktop', 'TFX_football_manager_2022', 'Artifacts')
SERVING_MODEL_DIR=os.path.join(os.sep, 'Users', 'CarlosMonsivais', 'Desktop', 'TFX_football_manager_2022', 'Model')
DATA_ROOT = '/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Data'

print('Artifact Storage: {}'.format(ARTIFACT_STORE))
print('Serving Model Storage: {}'.format(SERVING_MODEL_DIR))
print('Data Root: {}'.format(DATA_ROOT))

Artifact Storage: /Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts
Serving Model Storage: /Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Model
Data Root: /Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Data


# Pipeline Names

In [3]:
PIPELINE_NAME = 'tfx-football-pipeline'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))

# Creating a directory in the Artifact Storage variable where the pipline metadata will be stored using this timestamp.
os.makedirs(PIPELINE_ROOT, exist_ok = True)

print('Created a directory here:{} \nwhere the pipleine metadata artifacts will be stored.'.format(PIPELINE_ROOT))

Created a directory here:/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220114_155524 
where the pipleine metadata artifacts will be stored.


# Only for Jupyter Notebook Interactivity

In [4]:
context = InteractiveContext(pipeline_name = PIPELINE_NAME,
                             pipeline_root = PIPELINE_ROOT,
                             metadata_connection_config = None)

# Reading in Data

In [8]:
input_config = example_gen_pb2.Input(splits=[example_gen_pb2.Input.Split(name = 'football_manager_22', 
                                                                    pattern = '*.csv')
                                       ]
                               )

In [9]:
# Doing an 80:20 Train Eval Split
output_config = example_gen_pb2.Output(split_config = example_gen_pb2.SplitConfig(splits=[example_gen_pb2.SplitConfig.Split(name = 'train_data', hash_buckets = 4),
                                                                                          example_gen_pb2.SplitConfig.Split(name = 'test_data', hash_buckets = 1)
                                                                                         ]
                                                                                 )
                                      )

In [10]:
example_gen = tfx.components.CsvExampleGen(input_base = DATA_ROOT,
                                           input_config = input_config,
                                           output_config = output_config)

In [11]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 2
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 2
        type_id: 5
        uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220114_155524/CsvExampleGen/examples/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train_data\", \"test_data\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:football_manager_22,num_files:1,total_bytes:81478459,xor_checksum:1640463433,sum_checksum:1640463433"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [12]:
# Shows features
examples_uri = example_gen.outputs['examples'].get()[0].uri
tfrecord_filenames = [os.path.join(examples_uri, 'train_data', name)
                      for name in os.listdir(os.path.join(examples_uri, 'train_data'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

Wages: [b'\xe2\x82\xac 113,890 pw']
Teamwork: [16]
Work Rate: [15]
Pace: [12]
Vision: [15]
Length: [b'191 CM']
First Touch: [16]
Leadership: [14]
Reflexes: []
Long Shots: [15]
Kicking: []
Communication: []
Free Kick Taking: [15]
Age: [26]
Off the Ball: [16]
Long Throws: [3]
Flair: [18]
Finishing: [15]
Contract end: [b'2024-06-30']
Acceleration: [13]
Nation: [b'Serbia']
Strength: [17]
Roles: [b"[['Box to Box Midfielder (Support)', 'Central Midfielder (Attacking)', 'Mezzala (Attacking)', 'Advanced Playmaker (Support)']]"]
Decisions: [13]
Bravery: [17]
Corners: [3]
Determination: [17]
Unique ID: [62127037]
Technique: [16]
Name: [b'Sergej Milinkovic-Savic']
Ability: [82]
Penalty Taking: [13]
Passing: [16]
Heading: [17]
Marking: [9]
Position: [b'MC']
Caps / Goals: [b'22 / 3']
Natural Fitness: [20]
Anticipation: [15]
One on Ones: []
Jumping Reach: [17]
Crossing: [6]
Potential: [85]
Weight: [b'82 KG']
Concentration: [10]
Composure: [10]
Rushing Out (Tendency): []
Tackling: [13]
Punching (Tend

# StaisticsGen Component

In [13]:
statistics_gen = tfx.components.StatisticsGen(examples = example_gen.outputs['examples'])

In [14]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 3
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 3
        type_id: 7
        uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220114_155524/StatisticsGen/statistics/3"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train_data\", \"test_data\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [15]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


# SchemaGen

In [16]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                       infer_feature_shape=False)

In [17]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 4
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 9
        uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220114_155524/SchemaGen/schema/4"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [18]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Ability',INT,required,single,-
'Acceleration',INT,required,single,-
'Aerial Reach',INT,required,,-
'Age',INT,required,single,-
'Aggression',INT,required,single,-
...,...,...,...,...
'Unique ID',INT,required,single,-
'Vision',INT,required,single,-
'Wages',BYTES,required,,-


/opt/anaconda3/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Foot',"'Both', 'Left', 'Right'"
'Length',"'150 CM', '152 CM', '153 CM', '154 CM', '155 CM', '156 CM', '157 CM', '158 CM', '159 CM', '160 CM', '161 CM', '162 CM', '163 CM', '164 CM', '165 CM', '166 CM', '167 CM', '168 CM', '169 CM', '170 CM', '171 CM', '172 CM', '173 CM', '174 CM', '175 CM', '176 CM', '177 CM', '178 CM', '179 CM', '180 CM', '181 CM', '182 CM', '183 CM', '184 CM', '185 CM', '186 CM', '187 CM', '188 CM', '189 CM', '190 CM', '191 CM', '192 CM', '193 CM', '194 CM', '195 CM', '196 CM', '197 CM', '198 CM', '199 CM', '200 CM', '201 CM', '202 CM', '203 CM', '204 CM', '205 CM', '206 CM', '210 CM', '207 CM', '208 CM'"
'Weight',"'0 KG', '100 KG', '101 KG', '102 KG', '103 KG', '104 KG', '105 KG', '107 KG', '108 KG', '110 KG', '55 KG', '56 KG', '57 KG', '58 KG', '59 KG', '60 KG', '61 KG', '62 KG', '63 KG', '64 KG', '65 KG', '66 KG', '67 KG', '68 KG', '69 KG', '70 KG', '71 KG', '72 KG', '73 KG', '74 KG', '75 KG', '76 KG', '77 KG', '78 KG', '79 KG', '80 KG', '81 KG', '82 KG', '83 KG', '84 KG', '85 KG', '86 KG', '87 KG', '88 KG', '89 KG', '90 KG', '91 KG', '92 KG', '93 KG', '94 KG', '95 KG', '96 KG', '97 KG', '98 KG', '99 KG', '109 KG'"


## Updating SchemaGen File (In case you want to make any changes to the file.)

In [ ]:
# You can chage this file to customize it as you want it for your schema.
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

In [ ]:
# tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))   (for categorical variable)
tfdv.set_domain(schema, 'Ability',  schema_pb2.IntDomain(name='Ability', min=0, max=20))

In [ ]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

In [ ]:
schema_importer = ImporterNode(instance_name = 'Schema_Importer',
                               source_uri = schema_dir,
                               artifact_type=tfx.types.standard_artifacts.Schema,
                               reimport=False)

In [ ]:
context.run(schema_importer)

In [ ]:
context.show(schema_importer.outputs['result'])

# Example Validator (Anomaly Detection)

In [ ]:
example_validator = ExampleValidator(instance_name = "Data_Validation",
                                     statistics = statistics_gen.outputs['statistics'],
                                     schema = schema_importer.outputs['result'])

In [ ]:
context.run(example_validator)

In [ ]:
train_uri = example_validator.outputs['anomalies'].get()[0].uri
train_anomalies_filename = os.path.join(train_uri, "train/anomalies.pbtxt")

In [ ]:
train_anomalies_filename

In [ ]:
context.show(example_validator.outputs['output'])

# Transform